# Todo List
- modify dimension to RGB ✅
- capable of running CNN ✅
- Improve accuracy 👈
- fully shuffle the dataset 打乱数据集
- 保存特征
- 可视化网络


In [1]:
import os
import random
import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [2]:
"""
input dirctory of dataset
output ndarray of images and one-hot array of label
"""
def read_dataset(rootDir):
    print("loading "+rootDir)
    image_list = []
    label_list = []
    
    def load(rootDir):  
        for lists in os.listdir(rootDir): 
            path = os.path.join(rootDir, lists) 
            patharray = path.split("/")
            if (len(patharray) ==4) and (patharray[3]!=".DS_Store"):
                label = patharray[2]
                label_list.append(int(label)-1)
                raw = (skimage.data.imread(path))
                #在这里开始进行图形处理
                y = skimage.transform.resize(raw,(28,28))
                temp = y
                #处理到这里结束
                image_list.append(temp)
            if os.path.isdir(path): 
                load(path)
               
    load(rootDir)
    lable_tf = tf.one_hot(label_list,20)
    label_onehot = tf.Session().run(lable_tf)
    print ("load "+rootDir+" completed")
    return image_list, label_onehot

train_image, train_label = read_dataset("dataset/train")
test_image, test_label = read_dataset("dataset/test")
# print(train_image)
# print(train_label)



loading dataset/train
load dataset/train completed
loading dataset/test
load dataset/test completed


In [3]:
train_image = np.array(train_image, np.float32)
train_label = np.array(train_label, np.float32)
test_image = np.array(test_image, np.float32)
test_label = np.array(test_label, np.float32)



In [4]:
def shuffle_dataset(images, labels):   
    image_shuffle = []
    label_shuffle = []
    container = []
    for i in range(len(images)):
        container.append((images[i],labels[i]))
    random.shuffle(container)
    for i in range(len(images)):
        x,y= container[i]
        image_shuffle.append(x)
        label_shuffle.append(y)
    image_shuffle = np.array(image_shuffle, np.float32)
    label_shuffle = np.array(label_shuffle, np.float32)
    return image_shuffle, label_shuffle

In [5]:
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(v_ys, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')



In [6]:
# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 28,28,3])  # 28x28
ys = tf.placeholder(tf.float32, [None, 20])
keep_prob = tf.placeholder(tf.float32)

x_image = xs

## conv1 layer ##
W_conv1 = weight_variable([5, 5, 3, 32])  # patch 5x5, in size 1, out size 32
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)    # output size 32个28x28
h_pool1 = max_pool_2x2(h_conv1)  # output size 32个14x14

## conv2 layer ##
W_conv2 = weight_variable([5, 5, 32, 64])  # patch 5x5, in size 32, out size 64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  # output size 14x14x64
h_pool2 = max_pool_2x2(h_conv2)  # output size 64个7x7

## fc1 layer ##
h_pool2_flat = tf.reshape(h_pool2, [-1,3136])
W_fc1 = weight_variable([3136, 1024])
b_fc1 = bias_variable([1024])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## fc2 layer ##
W_fc2 = weight_variable([1024, 20])
b_fc2 = bias_variable([20])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# the error between prediction and real data
"""
交叉熵可在神经网络(机器学习)中作为损失函数，p表示真实标记的分布，q则为训练后的模型的预测标记分布，交叉熵损失函数可以衡量p与q的相似性。
交叉熵作为损失函数还有一个好处是使用sigmoid函数在梯度下降时能避免均方误差损失函数学习速率降低的问题，因为学习速率可以被输出的误差所控制。
"""
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),reduction_indices=[1]))  # loss
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)


In [7]:
sess = tf.Session()
if int((tf.__version__).split('.')[1]) < 12:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)

In [8]:
for i in range(20):
    
    train_image_shuffle, train_label_shuffle = shuffle_dataset(train_image,train_label)
    test_image_shuffle, test_label_shuffle = shuffle_dataset(test_image,test_label)

    train_image_batch1 = train_image_shuffle[0:120]
    train_image_batch2 = train_image_shuffle[120:240]
    train_image_batch3 = train_image_shuffle[240:360]
    train_image_batch4 = train_image_shuffle[360:480]
    train_image_batch5 = train_image_shuffle[480:600]
    train_label_batch1 = train_label_shuffle[0:120]
    train_label_batch2 = train_label_shuffle[120:240]
    train_label_batch3 = train_label_shuffle[240:360]
    train_label_batch4 = train_label_shuffle[360:480]
    train_label_batch5 = train_label_shuffle[480:600]

    sess.run(train_step, feed_dict={xs: train_image_batch1, ys: train_label_batch1, keep_prob: 0.5})
    sess.run(train_step, feed_dict={xs: train_image_batch2, ys: train_label_batch2, keep_prob: 0.5})
    sess.run(train_step, feed_dict={xs: train_image_batch3, ys: train_label_batch3, keep_prob: 0.5})
    sess.run(train_step, feed_dict={xs: train_image_batch4, ys: train_label_batch4, keep_prob: 0.5})
    sess.run(train_step, feed_dict={xs: train_image_batch5, ys: train_label_batch5, keep_prob: 0.5})
    
    accuracy = 100* compute_accuracy(test_image_shuffle, test_label_shuffle)
    print(accuracy)




4.2500000447
5.4999999702
4.7499999404
4.2500000447
4.50000017881
3.24999988079
6.49999976158
8.25000032783
7.24999979138
7.0000000298
7.24999979138
7.7500000596
8.50000008941
8.25000032783
9.25000011921
9.49999988079
10.2499999106
10.7500001788
11.249999702
10.7500001788


# Extracting feature
In this part, I use the parameter `W_conv1`,`b_conv1`,`W_conv2`,`b_conv2`,`W_fc1`,`b_fc1`,`W_fc2`,`b_fc2` to construct the model and use agian the training set to go through this network and extract the feature of this image.
## Getting the parameter of CNN

In [9]:
# print (W_conv1)
# print (b_conv1)
# print (W_conv2)
# print (b_conv2)
# print (W_fc1)
# print (b_fc1)
# print (W_fc2)
# print (b_fc2)

# W_conv11 = sess.run(W_conv1)

# print("======*======")
# print (W_conv11)

# print("======*======")
# sess.run(W_conv1.eval(session = sess))

From above we get the parameters that learn from CNN. The next step is use these parameters to construct a CNN

In [10]:
# def build_model ():
#     h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

# t-SNE Visulisation
In this part, I use t-SNE to visulise the